#### Flow matching

Goal: train NN $u_t^{\theta}(X_t)$ with parameters ${\theta}$ to approx the target marginal vector field $u_t^{target}(X_t)$. $u_t^{\theta}(X_t)\approx u_t^{target}(X_t)$

**Flow matching loss**:
$$
\begin{aligned}
& L_{fm}(\theta)=\mathbb{E}_{(t,z,x)} \left[||u_t^{\theta}(x)-u_t^{target}(x)||^2\right]
\\
& t\sim Unif[0,1]
\\
& z\sim p_{data}
\\
& x\sim p_t(\cdot|z)
\end{aligned}
$$

*However*, $u_t^{target}(x)=\int{u_t^{target}(x|z)\frac{p_t(x|z)p_{data}}{\int{p_t(x|z)p_{data}dz}}dz}$ is intractable/expensive to compute for every call to the loss fn when the size of the dataset is large.

**Conditional flow matching loss**:
$$
\begin{aligned}
& L_{cfm}(\theta)=\mathbb{E}_{(t,z,x)} \left[||u_t^{\theta}(x)-u_t^{target}(x|z)||^2\right]
\end{aligned}
$$

*Theorem:* $L_{fm}(\theta)=L_{cfm}(\theta)+C$, where $C$ is a constant independent of $\theta$. Hence,
1. Minimizer $\theta^*$ of $L_{cfm}$ is the marginal vector field $u_t^{\theta^*}=u_t^{target}$,
2. Gradients of the FM and CFM loss $\nabla_{\theta}L_{cfm}(\theta)=\nabla_{\theta}L_{fm}(\theta)$ hence SGD optimization of both is the same.

**Training algo**:

Given samples $z\sim p_{data}$ and NN $u_t^{\theta}$,
1. For each mini-batch do:
2.    _ Sample an example $z\sim p_{data}$
3.    _ Sample a random time $t\sim Unif[0,1]$
4.    _ Sample $x\sim p_t(\cdot|z)$
5.    _ Compute loss:
$$
\begin{aligned}
& L_{cfm}(\theta)=||u_t^{\theta}(x)-u_t^{target}(x|z)||^2
\end{aligned}
$$
6.    _ Update $\theta$ via gradient descent.

For a Gaussian *CondOT path*:

Given samples $z\sim p_{data}$ and NN $u_t^{\theta}$,
1. For each mini-batch do:
2.    _ Sample an example $z\sim p_{data}$
3.    _ Sample a random time $t\sim Unif[0,1]$
4.    _ Sample from a std Normal $\epsilon\sim\mathcal{N}(0, I_d)$
5.    _ $x=\alpha_t z+\beta_t \epsilon,  \alpha_t=t,  \beta_t=(1-t) \Rightarrow x=t(z-\epsilon)+\epsilon$
5.    _ Compute loss:
$$
\begin{aligned}
& L_{cfm}(\theta)=\left \Vert u_t^{\theta}(x)-\left[\left(\dot{\alpha_t}-\frac{\dot{\beta_t}}{\beta_t}\alpha_t\right)z+\frac{\dot{\beta_t}}{\beta_t}x\right]\right \Vert^2
\\
& L_{cfm}(\theta)=\left \Vert u_t^{\theta}(x)-\left[\left(1+\frac{1}{\beta_t}\alpha_t\right)z-\frac{1}{\beta_t}(\alpha_t z+\beta_t \epsilon)\right]\right \Vert^2
\\
& L_{cfm}(\theta)=\left \Vert u_t^{\theta}(t\left[z-\epsilon\right]+\epsilon)-\left[z-\epsilon\right]\right \Vert^2
\end{aligned}
$$
6.    _ Update $\theta$ via gradient descent.

Minimizing the loss function, the NN $u^\theta$ given a noisy image and $t$, $u_t^\theta(t\left[z-\epsilon\right]+\epsilon)$, learns to predict the difference between the original image and the noise vector added to it $(z-\epsilon)$.

$(z-\epsilon)$ is also the velocity at the current $t$ along the path from the noise vector to the sample. For paths other than the straight line CondOT path, this velocity would take a different (time-dependent) form.